In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import re
import json

In [6]:
benslog = "log.txt"

In [7]:
def read_data(filename):
    raw = []
    ttype_list = ['TRANSACTION', 'BLOCK', 'SOLVED']
    with open(filename, 'r') as f:
        for line in f:
            record = line.split(' ') # separating sign to be checked
            if len(record) > 2:
                if record[4] in ttype_list:
                    raw.append(record)

	# fileString = '{0:.6f}'.format(timestamp_a) + " " + str(self.name) + " " + str(status) + 
    # " " + str(bytes) + " " + str(ttype) + " " + str(tID) + " " + str(fromNode) + " " + 
    # str(toNode) + " " + str(sentTime) + "\n"
	# name: vm[#]node[i]
	# ttype: TRANSACTION/BLOCK/...etc
	# tID: Tx: txID/Block: selfHash/Puzzle: hash/Verify: hash_solution
    labels = ['timestamp', 'name', 'status', 'bytes', 'ttype', 'tID', 'fromNode', 'toNode', 'sentTime']
    df = pd.DataFrame.from_records(raw, columns=labels)
    return df

def read_blockTx(filename):
    raw = []
    with open(filename, 'r') as f:
        for line in f:
            record = line.split(' ') # separating sign to be checked
            if len(record) >= 2:
                raw.append(record)

    labels = ['blockHash', 'txID']
    df = pd.DataFrame.from_records(raw, columns=labels)
    return df

# def draw_hist(d, output_filename):
# 	"""
# 	d: array
# 	output_filename: filename without path
# 	"""
# 	output_path = "img/"
# 	output = output_path + output_filename

# 	sns.set(style="whitegrid")
# 	sns_plot = sns.distplot(d, kde=False, color="b")
# 	fig = sns_plot.get_figure()
# 	fig.savefig(output)
# 	plt.clf()

# def draw_line(df, x, y, color, output_filename):
# 	"""
# 	df: dataframe
# 	y: the column for y axis
# 	color: the grouping column
# 	output_filename: filename without path
# 	"""
# 	output_path = "img/"
# 	output = output_path + output_filename

# 	sns_plot = ""
# 	sns.set(style="whitegrid")
# 	sns.despine(left=True)
# 	# sns.set_palette(sns.color_palette("Blues"))
# 	if color != None:
# 		sns_plot = sns.lineplot(x=x, y=y, hue=color, data=df, legend=False)
# 	else:
# 		sns_plot = sns.lineplot(x=x, y=y, data=df, legend=False)
# 	fig = sns_plot.get_figure()
# 	fig.savefig(output)
# 	plt.clf()

In [13]:
raw_df20 = read_data(benslog)
raw_df20['nodeNum'] = 20
raw_df20['bytes'] = raw_df20['bytes'].astype(int)
raw_df20['timestamp'] = pd.to_datetime(raw_df20['timestamp'], unit='s')
raw_df20['sentTime'] = pd.to_datetime(raw_df20['sentTime'], unit='s')

raw_df = raw_df20.reset_index()

In [15]:
# blocks take how long to propagate to all nodes
block20_df = raw_df[raw_df.status == 'IncomingBlock' and raw_df.nodeNum == 20].sort_values(by=['timestamp']).drop_duplicates(subset=['tID', 'toNode'], keep="first")

# calculate the time elaspsed from solved to all nodes
block20_solved_df = raw_df[raw_df.ttype == 'SOLVED' and raw_df.nodeNum == 20].groupby(['tID'])['timestamp'].min().reset_index()

block20_solved = pd.Series(block20_solved_df.sentTime.values,index=block20_solved_df.tID).to_dict()

block20_df['sentTime'] = tx20_df.tID.apply(lambda x: block20_solved[x])

##### record the solved time
block20_df['timeElapsed'] = (block20_df.timestamp - block20_df.sentTime)

block20_df['timeElapsed'] = block20_df['timeElapsed'].dt.microseconds.abs()

# total elapsed time for each transaction forwarding
# histogram: x = timeElapsed
ttlTimeBlock20 = block20_df.groupby(['tID'])['timeElapsed'].max().reset_index()
ttlTimeBlock100 = block100_df.groupby(['tID'])['timeElapsed'].max().reset_index()
draw_hist(ttlTimeBlock20['timeElapsed'].values, 'plot09_hist_block_propagation_delay_all_20.png')
draw_hist(ttlTimeBlock100['timeElapsed'].values, 'plot10_hist_block_propagation_delay_all_100.png')

# Plot 11 & 12
# Splits, how often, the longest split (height)
# scatter: x=time, y=split height


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().